In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
import base64
from dash.exceptions import PreventUpdate
from pymongo import MongoClient
from bson.json_util import dumps

from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "accuser"
password = "mysafepassword"
shelter = AnimalShelter()


# Types of rescue dogs based on provided spec sheet
# Using age_upon_outcome_in_weeks for age query as suggested in spec sheet
df = pd.DataFrame.from_records(shelter.read_all({}))
dfwr = pd.DataFrame.from_records(shelter.read_all({"animal_type":"Dog","breed":
             {"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
             "sex_upon_outcome":"Intact Female",
             "age_upon_outcome_in_weeks":{"$gte":26},
             "age_upon_outcome_in_weeks":{"$lte":156}}))
dfmwr = pd.DataFrame.from_records(shelter.read_all({"animal_type":"Dog","breed":
             {"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Huskey","Rottweiler"]},
             "sex_upon_outcome":"Intact Male",
             "age_upon_outcome_in_weeks":{"$gte":26},
             "age_upon_outcome_in_weeks":{"$lte":156}}))
dfdit = pd.DataFrame.from_records(shelter.read_all({"animal_type":"Dog","breed":
             {"$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},
             "sex_upon_outcome":"Intact Female",
             "age_upon_outcome_in_weeks":{"$gte":20},
             "age_upon_outcome_in_weeks":{"$lte":300}}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'graziososalvare.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
#   html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Center(
    html.A(
        [html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), alt='Grazioso Salvare Logo',
            style={'height': '30%', 'width':'30%'})], href='https://www.snhu.edu'
    )),
    html.H6('Sergio Udave'),
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'WR'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'MWR'},
                {'label': 'Disaster or Individual Tracking', 'value': 'DIT'},
                {'label': 'Reset', 'value':'R'}
            ],
            value='R',
            labelStyle={'display': 'inline-block'}
            )
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'), 
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
        
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################   
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
        # Filter type compared with previously defined values for
        # radio items (Water Rescue, Reset, etc.)
        if filter_type == 'R':
            new_df = df
        elif filter_type == 'WR':
            new_df = dfwr
        elif filter_type == 'MWR':
            new_df = dfmwr
        elif filter_type == 'DIT':
            new_df = dfdit
        else:
            new_df = df
            
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data = new_df.to_dict('records')
        
        return (data,columns)


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
        dff = pd.DataFrame.from_dict(viewData)
        new_df = dff.breed.value_counts()
        new_df.name = "breed"
        new_df = new_df.to_frame()
        new_df.reset_index(inplace=True)
         
        return [
        dcc.Graph(
        figure = px.pie(new_df, values="breed", names="index"))
        ]
   
    
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_map(viewData):
    viewDF = pd.DataFrame.from_dict(viewData)
    dff = viewDF.loc[0]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker placement based on latitude and longitude
            dl.Marker(position=[
                dff.loc['location_lat'], 
                dff.loc['location_long'],
            ],
                     # Tooltip and Popup based on breed and name
                     children=[
                         dl.Tooltip(dff['breed']),
                         dl.Popup([html.H2("Animal Name"), 
                                   html.P(dff.loc['name'])
                                  ])
                     ])
        ])
    ]


app